In [6]:
import pandas as pd

In [7]:
# Load dataset
file_path = 'Online Retail-unclean.xlsx'  # Ganti dengan path file Anda
data = pd.read_excel(file_path)

### cek data missing

In [8]:
missing_summary = data.isnull().sum()
print("Jumlah missing data per kolom:")
print(missing_summary)

# Filter baris yang memiliki missing values
missing_rows = data[data.isnull().any(axis=1)]
print("\nBaris dengan missing data:")
print(missing_rows)

Jumlah missing data per kolom:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           5
CustomerID     135086
Country             8
dtype: int64

Baris dengan missing data:
       InvoiceNo StockCode                      Description  Quantity  \
205       536389     22726       ALARM CLOCK BAKELIKE GREEN         4   
622       536414     22139                              NaN        56   
1443      536544     21773  DECORATIVE ROSE BATHROOM BOTTLE         1   
1444      536544     21774  DECORATIVE CATS BATHROOM BOTTLE         2   
1445      536544     21786               POLKADOT RAIN HAT          4   
...          ...       ...                              ...       ...   
541536    581498    85099B          JUMBO BAG RED RETROSPOT         5   
541537    581498    85099C   JUMBO  BAG BAROQUE BLACK WHITE         4   
541538    581498     85150    LADIES & GENTLEMEN METAL SIGN         1   
541539    581498     8

### Mengisi missing data pada kolom 'Description' berdasarkan 'StockCode'

In [9]:
# Buat dictionary untuk mapping StockCode -> Description
description_mapping = data.dropna(subset=['Description']).set_index('StockCode')['Description'].to_dict()

# Isi missing values di 'Description' berdasarkan StockCode
data['Description'] = data.apply(
    lambda row: description_mapping.get(row['StockCode'], 'Unknown') if pd.isnull(row['Description']) else row['Description'],
    axis=1
)


### Kolom numerik seperti UnitPrice bisa diisi dengan disamakan dengan stockcode yang sama

In [10]:
data['UnitPrice'] = data['UnitPrice'].fillna(data['UnitPrice'].median())

### 2. Mengisi missing data pada 'CustomerID' berdasarkan 'InvoiceNo'

In [11]:
# Buat dictionary untuk mapping InvoiceNo -> CustomerID
customer_id_mapping = data.dropna(subset=['CustomerID']).set_index('InvoiceNo')['CustomerID'].to_dict()

# Isi missing values di 'CustomerID' berdasarkan InvoiceNo
data['CustomerID'] = data.apply(
    lambda row: customer_id_mapping.get(row['InvoiceNo'], 0) if pd.isnull(row['CustomerID']) else row['CustomerID'],
    axis=1
)

### 3. Mengisi missing data pada 'Country' berdasarkan 'CustomerID'

In [12]:
# Buat dictionary untuk mapping CustomerID -> Country
country_mapping = data.dropna(subset=['Country']).set_index('CustomerID')['Country'].to_dict()

# Isi missing values di 'Country' berdasarkan CustomerID
data['Country'] = data.apply(
    lambda row: country_mapping.get(row['CustomerID'], 'Unknown') if pd.isnull(row['Country']) else row['Country'],
    axis=1
)

In [13]:
unit_price_mapping = data.dropna(subset=['UnitPrice']).set_index('StockCode')['UnitPrice'].to_dict()
data['UnitPrice'] = data.apply(
    lambda row: unit_price_mapping.get(row['StockCode'], row['UnitPrice']) if pd.isnull(row['UnitPrice']) else row['UnitPrice'],
    axis=1
)

# Simpan Data Bersih
## Setelah membersihkan data, simpan ke file Excel baru:

In [14]:
# Save hasil cleaning ke file baru
cleaned_file_path = '/mnt/data/Online Retail-Fully-Cleaned.xlsx'
data.to_excel(cleaned_file_path, index=False)

print(f"Data cleaned and saved to: {cleaned_file_path}")

OSError: Cannot save file into a non-existent directory: '\mnt\data'